# Homework (12 pts) - Cross Validation

## Nisha Dalal nd8288

In [7]:
import numpy as np
import scipy as sp
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'retina'

In [8]:
from sklearn import datasets

diabetes = datasets.load_diabetes()

print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

In [9]:
# features (X) are already mean subtracted and normalized
X = diabetes.data
# target disease progression values are NOT mean subtraced or normalized
y = diabetes.target

1. (3 pts) Split dataset into training (80%) and testing (20%) sets.

In [12]:
X = df.loc[:,'AGE':'GLU'].to_numpy()
y = df['DIS'].to_numpy()

featureMeans = X.mean(axis=0)
featureStd = X.std(axis=0)

Xs = (X - featureMeans) / featureStd

disMean = y.mean()
disStd = y.std()

ys = (y - disMean) / disStd

NameError: name 'df' is not defined

In [11]:


# exact way data are split is determined by random_state
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, shuffle=True, test_size=0.5, random_state=1)
alphas = np.logspace(-5, 2, 50)
mse = np.zeros(alphas.shape)
for i, alpha in enumerate(alphas):
    model = Ridge(alpha=alpha)
    model.fit(Xs_train, ys_train)
    residuals = ys_test - model.predict(Xs_test)
    mse[i] = np.sum(residuals**2) / len(residuals)
    
ind = np.argmin(mse)
alpha = alphas[ind]
plt.plot(alphas, mse, 'o-')
plt.xscale('log')
plt.xlabel(r'$\alpha$')
plt.ylabel('Ridge Test MSE')
plt.title(f"Best alpha = {alpha}");

NameError: name 'Xs' is not defined

2. (3 pts) Use Ridge regression and 5-fold cross validation to model your training set.

3. (3 pts) Use Lasso regression and 5-fold cross validation to model your training set.

4. (3 pts) Compare the Ridge and Lasso models using your testing set. Which model is preferred?